In [1]:
!pip install -q autogluon.tabular ray==2.10.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
aiobotocore 2.15.0 requires botocore<1.35.17,>=1.35.16, but you have botocore 1.29.165 which is incompatible.
bigframes 0.22.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.10.0, but you have google-cloud-bigquery 2.34.4 which is incompatible.
bigframes 0.22.0 requires google-cloud-storage>=2.0.0, but you have google-cloud-storage 1.44.0 which is incompatible.
bigframes 0.22.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.2 which is incompatible.
cesium 0.12.3 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
dataproc-jupyter-plugin 0.1.79 requires pydantic~=1.10.0, but you have pydantic 2.9.2 which is incompatible.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.
libp

In [2]:
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.base import clone
import pandas as pd
import numpy as np
import warnings
import pickle
import shutil
import gc

warnings.filterwarnings('ignore')

In [3]:
class CFG:
    train_path = '/kaggle/input/playground-series-s4e10/train.csv'
    test_path = '/kaggle/input/playground-series-s4e10/test.csv'
    sample_sub_path = '/kaggle/input/playground-series-s4e10/sample_submission.csv'
    
    target = 'loan_status'
    n_folds = 5
    seed = 1859
    time_limit = 3600 * 10

In [4]:
train = pd.read_csv(CFG.train_path, index_col='id')
test = pd.read_csv(CFG.test_path, index_col='id')

In [5]:
def save_pred_probs(pred_probs, cv_score, name):
    with open(f'autogluon_{name}_pred_probs_{cv_score:.6f}.pkl', 'wb') as f:
        pickle.dump(pred_probs, f)

def save_submission(test_pred_probs, score):
    sub = pd.read_csv(CFG.sample_sub_path)
    sub[CFG.target] = test_pred_probs[:, 1]
    sub.to_csv(f'sub_autogluon_{score:.6f}.csv', index=False)

In [6]:
scores = []
leaderboards = []
oof_pred_probs = np.zeros((train.shape[0], len(np.unique(train[CFG.target]))))
test_pred_probs = np.zeros((test.shape[0], len(np.unique(train[CFG.target]))))

skf = StratifiedKFold(n_splits=CFG.n_folds, random_state=CFG.seed, shuffle=True)
split = skf.split(train.drop(columns=CFG.target), train[CFG.target])
for fold_idx, (train_idx, val_idx) in enumerate(split):
    _train, _val = train.iloc[train_idx], train.iloc[val_idx]   
    
    predictor = TabularPredictor(
        label=CFG.target,
        eval_metric='roc_auc',
        problem_type='binary',
        verbosity=2
    )
    
    predictor.fit(
        train_data=_train,
        time_limit=CFG.time_limit // CFG.n_folds,
        presets='best_quality',
        ag_args_fit={
            'num_gpus': 1, 
            'num_cpus': 4
        }
    )

    y_pred_probs = predictor.predict_proba(_val).values
    oof_pred_probs[val_idx] = y_pred_probs            

    temp_test_pred_probs = predictor.predict_proba(test).values
    test_pred_probs += temp_test_pred_probs / CFG.n_folds

    score = roc_auc_score(_val[CFG.target], y_pred_probs[:, 1])
    scores.append(score)
    
    leaderboards.append(predictor.leaderboard(silent=True))

    del predictor, _train, _val, y_pred_probs, temp_test_pred_probs
    gc.collect()

No path specified. Models will be saved in: "AutogluonModels/ag-20241002_063738"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.14
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Jun 27 20:43:36 UTC 2024
CPU Count:          4
Memory Avail:       30.20 GB / 31.36 GB (96.3%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets o

(_ray_fit pid=407) [1000]	valid_set's binary_logloss: 0.166205


(_ray_fit pid=455) 	Training S1F3 with GPU, note that this may negatively impact model quality compared to CPU training.
(_ray_fit pid=503) 	Training S1F4 with GPU, note that this may negatively impact model quality compared to CPU training.


(_ray_fit pid=503) [1000]	valid_set's binary_logloss: 0.16453


(_ray_fit pid=551) 	Training S1F5 with GPU, note that this may negatively impact model quality compared to CPU training.
(_ray_fit pid=599) 	Training S1F6 with GPU, note that this may negatively impact model quality compared to CPU training.
(_ray_fit pid=647) 	Training S1F7 with GPU, note that this may negatively impact model quality compared to CPU training.
(_ray_fit pid=695) 	Training S1F8 with GPU, note that this may negatively impact model quality compared to CPU training.
(_dystack pid=165) 	0.9382	 = Validation score   (roc_auc)
(_dystack pid=165) 	76.24s	 = Training   runtime
(_dystack pid=165) 	1.12s	 = Validation runtime
(_dystack pid=165) Fitting model: LightGBM_BAG_L1 ... Training model for up to 1113.7s of the 1712.42s of remaining time.
(_dystack pid=165) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (1 workers, per: cpus=4, gpus=1, memory=0.07%)
(_ray_fit pid=749) 	Training S1F1 with GPU, note that this may negatively impact model

...

(_ray_fit pid=4988) /opt/conda/lib/python3.10/site-packages/fastai/learner.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
(_ray_fit pid=4988)   state = torch.load(file, 

(autoscaler +29m20s) Warning: The following resource request cannot be scheduled right now: {'CPU': 4.0, 'GPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.


(_ray_fit pid=5104) /opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [07:06:59] WARNING: /workspace/src/common/error_msg.cc:45: `gpu_id` is deprecated since2.0.0, use `device` instead. E.g. device=cpu/cuda/cuda:0
(_ray_fit pid=5104)   warnings.warn(smsg, UserWarning)
(_ray_fit pid=5104) /opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [07:06:59] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.
(_ray_fit pid=5104) 
(_ray_fit pid=5104)     E.g. tree_method = "hist", device = "cuda"
(_ray_fit pid=5104) 
(_ray_fit pid=5104)   warnings.warn(smsg, UserWarning)
(_ray_fit pid=5104) /opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [07:07:00] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

(autoscaler +29m55s) Warning: The following resource request cannot be scheduled right now: {'CPU': 4.0, 'GPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.


(_ray_fit pid=5565) 	Training S1F2 with GPU, note that this may negatively impact model quality compared to CPU training.
(_ray_fit pid=5565) 	Ran out of time, early stopping on iteration 47. Best iteration is:
(_ray_fit pid=5565) 	[47]	valid_set's binary_logloss: 0.175878
(_ray_fit pid=5613) 	Training S1F3 with GPU, note that this may negatively impact model quality compared to CPU training.
(_ray_fit pid=5613) 	Ran out of time, early stopping on iteration 64. Best iteration is:
(_ray_fit pid=5613) 	[64]	valid_set's binary_logloss: 0.169123
(_ray_fit pid=5661) 	Training S1F4 with GPU, note that this may negatively impact model quality compared to CPU training.
(_ray_fit pid=5661) 	Ran out of time, early stopping on iteration 53. Best iteration is:
(_ray_fit pid=5661) 	[53]	valid_set's binary_logloss: 0.171444
(_ray_fit pid=5709) 	Training S1F5 with GPU, note that this may negatively impact model quality compared to CPU training.
(_ray_fit pid=5709) 	Ran out of time, early stopping on 

In [7]:
for i in range(CFG.n_folds):
    print(f'--- Fold {i + 1} - AUC: {scores[i]:.6f}')

overall_score = roc_auc_score(train[CFG.target], oof_pred_probs[:, 1])
print(f'\n------ Overall: {overall_score:.6f} | Average: {np.mean(scores):.6f} ± {np.std(scores):.6f}')

--- Fold 1 - AUC: 0.963674
--- Fold 2 - AUC: 0.960475
--- Fold 3 - AUC: 0.960544
--- Fold 4 - AUC: 0.957793
--- Fold 5 - AUC: 0.960270

------ Overall: 0.960483 | Average: 0.960551 ± 0.001868


In [8]:
save_pred_probs(oof_pred_probs, np.mean(scores), 'oof')
save_pred_probs(test_pred_probs, np.mean(scores), 'test')
save_submission(test_pred_probs, np.mean(scores)) 

In [9]:
leaderboards[0].style.background_gradient(subset=['score_val'], cmap='RdYlGn')

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,0.959745,roc_auc,36.881506,4861.357508,0.007046,4.088954,3,True,36
1,LightGBM_r131_BAG_L2,0.958659,roc_auc,29.626605,3603.068275,0.662304,81.369918,2,True,34
2,LightGBMXT_BAG_L2,0.958591,roc_auc,29.195756,3566.826646,0.231456,45.128288,2,True,21
3,LightGBM_BAG_L2,0.958482,roc_auc,29.140867,3565.223204,0.176567,43.524846,2,True,22
4,WeightedEnsemble_L2,0.958216,roc_auc,3.062087,495.970657,0.007174,3.181508,2,True,20
5,XGBoost_BAG_L2,0.958152,roc_auc,29.170732,3546.662780,0.206431,24.964422,2,True,29
6,CatBoost_BAG_L2,0.958066,roc_auc,29.052873,3704.105826,0.088573,182.407468,2,True,25
7,CatBoost_r177_BAG_L2,0.957895,roc_auc,29.030289,3635.903119,0.065989,114.204762,2,True,32
8,LightGBMLarge_BAG_L2,0.957886,roc_auc,29.169819,3586.865589,0.205519,65.167231,2,True,31
9,NeuralNetTorch_r79_BAG_L2,0.957418,roc_auc,29.586313,3901.804540,0.622012,380.106182,2,True,33


In [10]:
leaderboards[1].style.background_gradient(subset=['score_val'], cmap='RdYlGn')

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,0.960649,roc_auc,24.776467,4873.768945,0.007301,4.089004,3,True,36
1,LightGBMXT_BAG_L2,0.959535,roc_auc,17.240424,3596.182907,0.204898,44.013615,2,True,21
2,LightGBM_r131_BAG_L2,0.959414,roc_auc,17.509356,3626.159512,0.473831,73.990220,2,True,34
3,XGBoost_BAG_L2,0.959166,roc_auc,17.239175,3577.417972,0.203649,25.248680,2,True,29
4,NeuralNetFastAI_BAG_L2,0.959128,roc_auc,17.863782,3999.248523,0.828256,447.079231,2,True,28
5,WeightedEnsemble_L2,0.959099,roc_auc,3.077857,516.223255,0.007283,3.106102,2,True,20
6,CatBoost_BAG_L2,0.959086,roc_auc,17.114754,3701.544541,0.079228,149.375249,2,True,25
7,LightGBM_BAG_L2,0.959008,roc_auc,17.175800,3594.630725,0.140274,42.461433,2,True,22
8,CatBoost_r177_BAG_L2,0.958773,roc_auc,17.104766,3649.333369,0.069240,97.164077,2,True,32
9,LightGBMLarge_BAG_L2,0.958229,roc_auc,17.238772,3619.143510,0.203246,66.974218,2,True,31


In [11]:
leaderboards[2].style.background_gradient(subset=['score_val'], cmap='RdYlGn')

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,0.961205,roc_auc,28.875513,4998.026376,0.007069,4.118610,3,True,36
1,LightGBMXT_BAG_L2,0.959902,roc_auc,19.858359,3594.120396,0.222562,47.101120,2,True,21
2,WeightedEnsemble_L2,0.959820,roc_auc,3.911530,645.382606,0.007357,3.245465,2,True,20
3,CatBoost_BAG_L2,0.959660,roc_auc,19.721283,3703.141599,0.085486,156.122324,2,True,25
4,LightGBM_r131_BAG_L2,0.959648,roc_auc,20.139326,3621.201132,0.503529,74.181856,2,True,34
5,CatBoost_r177_BAG_L2,0.959565,roc_auc,19.704864,3655.988770,0.069067,108.969495,2,True,32
6,LightGBM_BAG_L2,0.959484,roc_auc,19.771065,3591.820426,0.135268,44.801150,2,True,22
7,XGBoost_BAG_L2,0.959353,roc_auc,19.854874,3573.063572,0.219077,26.044296,2,True,29
8,LightGBMLarge_BAG_L2,0.959043,roc_auc,19.829057,3609.627076,0.193260,62.607800,2,True,31
9,NeuralNetTorch_r79_BAG_L2,0.958968,roc_auc,20.270377,3895.500980,0.634580,348.481704,2,True,33


In [12]:
leaderboards[3].style.background_gradient(subset=['score_val'], cmap='RdYlGn')

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,0.960983,roc_auc,26.834642,5005.763803,0.007380,4.237557,3,True,36
1,LightGBMXT_BAG_L2,0.959918,roc_auc,20.841488,3591.004287,0.199102,44.863771,2,True,21
2,WeightedEnsemble_L2,0.959795,roc_auc,3.638336,661.772497,0.007268,3.167202,2,True,20
3,CatBoost_BAG_L2,0.959712,roc_auc,20.721673,3693.974344,0.079287,147.833828,2,True,25
4,LightGBM_r131_BAG_L2,0.959552,roc_auc,21.144093,3621.761664,0.501706,75.621148,2,True,34
5,CatBoost_r177_BAG_L2,0.959506,roc_auc,20.716485,3657.753899,0.074099,111.613383,2,True,32
6,XGBoost_BAG_L2,0.959119,roc_auc,20.846380,3571.032619,0.203993,24.892103,2,True,29
7,LightGBM_BAG_L2,0.959085,roc_auc,20.788730,3592.267743,0.146343,46.127227,2,True,22
8,NeuralNetFastAI_BAG_L2,0.959026,roc_auc,21.446567,3969.916031,0.804181,423.775515,2,True,28
9,LightGBMLarge_BAG_L2,0.958855,roc_auc,20.845651,3609.627569,0.203265,63.487053,2,True,31


In [13]:
leaderboards[4].style.background_gradient(subset=['score_val'], cmap='RdYlGn')

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,0.960457,roc_auc,33.331230,4834.965731,0.007381,4.143406,3,True,36
1,LightGBMXT_BAG_L2,0.959490,roc_auc,25.792334,3604.371797,0.216920,46.593970,2,True,21
2,LightGBM_r131_BAG_L2,0.959374,roc_auc,26.168948,3634.598447,0.593534,76.820620,2,True,34
3,XGBoost_BAG_L2,0.959059,roc_auc,25.767199,3582.660808,0.191784,24.882982,2,True,29
4,WeightedEnsemble_L2,0.958967,roc_auc,3.848727,872.283406,0.007074,3.186055,2,True,20
5,CatBoost_BAG_L2,0.958933,roc_auc,25.655411,3716.684421,0.079997,158.906595,2,True,25
6,CatBoost_r177_BAG_L2,0.958856,roc_auc,25.647465,3663.982291,0.072051,106.204465,2,True,32
7,LightGBM_BAG_L2,0.958557,roc_auc,25.714019,3601.979158,0.138605,44.201331,2,True,22
8,LightGBMLarge_BAG_L2,0.958284,roc_auc,25.768099,3623.332653,0.192685,65.554826,2,True,31
9,NeuralNetFastAI_BAG_L2,0.958126,roc_auc,26.392796,3994.289176,0.817382,436.511350,2,True,28


In [14]:
shutil.rmtree("AutogluonModels")